# 爬取数据

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import pandas as pd
import time


# 配置Chrome选项
options = Options()
options.headless = False  # 设置为True时不会打开浏览器界面，设置为False可以看到浏览器

# 设置Chrome驱动
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Step 1: 打开搜索页面
driver.get("https://racing.on.cc/cgi-bin/srh/search/search.cgi")
time.sleep(2)  # 等待页面加载

# Step 2: 找到“赛事年度（开始年份）”选择框并选择"2024"
start_year_select = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "fryear"))  # 根据实际的字段名称来获取
)
start_year_select = Select(start_year_select)
start_year_select.select_by_visible_text("2023")  # 选择2024

# Step 3: 找到“赛事年度（结束年份）”选择框并选择"2025"
end_year_select = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "toyear"))  # 根据实际的字段名称来获取
)
end_year_select = Select(end_year_select)
end_year_select.select_by_visible_text("2023")  # 选择2025

# Step 4: 找到提交按钮，先使用滚动和显式等待确保按钮可点击
submit_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.NAME, "submitButton"))
)

# 确保按钮可见
driver.execute_script("arguments[0].scrollIntoView();", submit_button)

# 点击按钮
submit_button.click()  # 提交表单

# Step 5: 等待页面跳转到结果页面
time.sleep(3)  # 等待页面跳转

# Step 6: 在结果页面中找到链接文本为"192"的元素并点击
try:
    # 根据链接文本来找到并点击
    link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.LINK_TEXT, "7360"))  # 查找链接文本为"192"的元素
    )
    link.click()  # 点击链接

    # 等待页面跳转
    time.sleep(3)  # 等待页面加载到目标页面

    # Step 7: 获取目标页面中的数据
    print("Current URL:", driver.current_url)  # 打印当前页面URL，确保跳转正确

    # 获取表格内容
    try:
        # 使用提供的XPath定位表格
        table = driver.find_element(By.XPATH, '//*[@id="maintable"]/tbody/tr[5]/td/table/tbody/tr/td[2]/table/tbody/tr[4]/td[2]/table/tbody/tr/td/table/tbody/tr[2]/td[2]/table')
        rows = table.find_elements(By.TAG_NAME, "tr")  # 获取所有行

        # 数据存储列表
        table_data = []

        # 遍历每一行，输出数据
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")  # 获取每行的所有单元格
            cell_text = [cell.text.strip() if cell.text.strip() != "" else "" for cell in cells]  # 如果单元格为空，插入空字符串
            table_data.append(cell_text)  # 将数据添加到列表中

        # 检查是否获取到数据
        if table_data:
            # 将数据转换为DataFrame
            df = pd.DataFrame(table_data)

            # 保存为CSV文件，使用 utf-8-sig 编码，确保中文字符在Excel中正常显示
            df.to_csv("/Users/Keanu/Desktop/racing_data2.csv", index=False, encoding="utf-8")

            print("数据已保存为racing_data.csv")
        else:
            print("没有获取到表格数据。")
    except Exception as e:
        print(f"Error extracting table data: {e}")

except Exception as e:
    print(f"Error clicking the link: {e}")

# Step 8: 关闭浏览器
driver.quit()


# 爬取数据 New Version

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random

# 配置Chrome选项
options = Options()
options.headless = False  # 设置为True时不会打开浏览器界面，设置为False可以看到浏览器

# 设置Chrome驱动
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Step 1: 打开搜索页面
    driver.get("https://racing.on.cc/cgi-bin/srh/search/search.cgi")
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.NAME, "fryear"))
    )

    # Step 2: 找到“赛事年度（开始年份）”选择框并选择"2024"
    start_year_select = Select(driver.find_element(By.NAME, "fryear"))
    start_year_select.select_by_visible_text("2023")

    # Step 3: 找到“赛事年度（结束年份）”选择框并选择"2025"
    end_year_select = Select(driver.find_element(By.NAME, "toyear"))
    end_year_select.select_by_visible_text("2023")

    # Step 4: 找到提交按钮并点击
    submit_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.NAME, "submitButton"))
    )
    driver.execute_script("arguments[0].scrollIntoView();", submit_button)
    driver.execute_script("arguments[0].click();", submit_button)  # 强制使用JavaScript点击按钮

    # Step 5: 等待页面跳转到结果页面
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.LINK_TEXT, "7360"))
    )

    # Step 6: 点击结果页面的链接
    link = driver.find_element(By.LINK_TEXT, "7360")
    link.click()

    # Step 7: 获取目标页面中的数据
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="maintable"]/tbody/tr[5]/td/table'))
    )
    table = driver.find_element(By.XPATH, '//*[@id="maintable"]/tbody/tr[5]/td/table/tbody/tr/td[2]/table/tbody/tr[4]/td[2]/table/tbody/tr/td/table/tbody/tr[2]/td[2]/table')
    rows = table.find_elements(By.TAG_NAME, "tr")

    # 数据存储列表
    table_data = []
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        cell_text = [cell.text.strip() if cell.text.strip() != "" else "" for cell in cells]
        table_data.append(cell_text)

    # 保存数据到CSV文件
    if table_data:
        df = pd.DataFrame(table_data)
        df.to_csv("/Users/Keanu/Desktop/racing_data2.csv", index=False, encoding="utf-8")
        print("数据已保存为 racing_data2.csv")
    else:
        print("没有获取到表格数据。")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Step 8: 关闭浏览器
    driver.quit()


# Step1.将四个从网页上爬取下来的数据集进行整合

In [ ]:
import pandas as pd

# File paths
file_paths = [
    '/Users/Keanu/Desktop/racing_data/csv/冠.csv',
    '/Users/Keanu/Desktop/racing_data/csv/亚.csv',
    '/Users/Keanu/Desktop/racing_data/csv/季.csv',
    '/Users/Keanu/Desktop/racing_data/csv/负.csv'
]

# Load each CSV file into a DataFrame and concatenate them
combined_all_df = pd.concat([pd.read_csv(file) for file in file_paths], axis=0, ignore_index=True).drop_duplicates()

# Set output file path and export the combined DataFrame to CSV
output_file_path = '/Users/Keanu/Desktop/all.csv'
combined_all_df.to_csv(output_file_path, index=False, encoding='utf-8')

print(f"The combined table has been saved to: '{output_file_path}'")


# Step2.按照日期和场次排好

In [ ]:
import pandas as pd

# Load the CSV file
file_path = '/Users/Keanu/Desktop/all.csv'
data = pd.read_csv(file_path)

# Convert '日期' column to datetime
data['日期'] = pd.to_datetime(data['日期'], format='%m/%d/%y', errors='coerce')

# Drop rows with invalid dates
data = data.dropna(subset=['日期'])

# Sort by '日期' in descending order
df_sorted = data.sort_values(by='日期', ascending=False)

# Sort by '場' in descending order while preserving the order of '日期'
df_sorted = df_sorted.sort_values(by=['日期', '場'], ascending=[False, False])

# Reset the index
df_sorted.reset_index(drop=True, inplace=True)

# Save the sorted data to a new CSV file
df_sorted.to_csv('/Users/Keanu/Desktop/all_sorted.csv', index=False)

print("Data has been sorted and saved as 'all_sorted.csv'")


# Step3.检查缺失值

In [ ]:
import pandas as pd

# Load the CSV file
file_path = '/Users/Keanu/Desktop/all_sorted.csv'
data = pd.read_csv(file_path)

# Check for missing values in each column
missing_values = data.isnull().sum()
print("Missing values in each column:\n", missing_values)

# Extract rows with missing values
rows_with_missing = data[data.isnull().any(axis=1)]

# Save the rows with missing values to a new CSV file
rows_with_missing.to_csv('/Users/Keanu/Desktop/rows_with_missing_values.csv', index=False)

print("Rows with missing values have been saved as 'rows_with_missing_values.csv'")


# Step4.数据清洗

In [ ]:
import pandas as pd

# Load the CSV file
file_path = '/Users/Keanu/Desktop/all_sorted.csv'
data = pd.read_csv(file_path)

# Replace missing values in '配備' column with 'None'
data['配備'].fillna('None', inplace=True)

# Drop rows with any other missing values
data = data.dropna()

# Remove rows where '名' column is equal to 90
data = data[data['名'] != 90]

# Check for missing values in each column
missing_values = data.isnull().sum()
print("Missing values in each column:\n", missing_values)

# Save the cleaned data to a new CSV file
data.to_csv('/Users/Keanu/Desktop/all_sorted_cleaned.csv', index=False)

print("Cleaned data has been saved as 'all_sorted_cleaned.csv'")

